In [ ]:
pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

MessageError: Error: credential propagation was unsuccessful

In [ ]:
from google.colab import files

# kaggle.json 파일 업로드
files.upload()


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
import kaggle
!kaggle competitions download -c 6th-aikuthon -p /content/drive/MyDrive/Kaggle/6th-aikuthon


In [ ]:
!unzip /content/drive/MyDrive/Kaggle/6th-aikuthon/6th-aikuthon.zip -d /content/drive/MyDrive/Kaggle/6th-aikuthon/


Archive:  /content/drive/MyDrive/Kaggle/6th-aikuthon/6th-aikuthon.zip
  inflating: /content/drive/MyDrive/Kaggle/6th-aikuthon/sample_submission.csv  
  inflating: /content/drive/MyDrive/Kaggle/6th-aikuthon/test.json  
  inflating: /content/drive/MyDrive/Kaggle/6th-aikuthon/train.json  


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
import json
# json 파일 경로
file_path_train = '/content/drive/MyDrive/Kaggle/6th-aikuthon/train.json'
file_path_test = '/content/drive/MyDrive/Kaggle/6th-aikuthon/test.json'

# json 파일 열기 및 파싱
trainset = pd.read_json(file_path_train)
trainset = pd.read_json(file_path_test)

# 데이터 확인
print(trainset)
print(testset)


NameError: name 'pd' is not defined

In [ ]:
trainset[0]

{'info': {'caseField': '1',
  'detailField': '1',
  'trailField': '2',
  'caseNm': '대여금',
  'courtNm': '울산지방법원',
  'judmnAdjuDe': '2021. 05. 25.',
  'relateLaword': [],
  'qotatPrcdnt': []},
 'concerned': {'acusr': '1', 'dedat': '1'},
 'org': {'orgJdgmnCourtNm': '울산지방법원',
  'orgJdgmnAdjuDe': '2019. 7. 11. ',
  'orgJdgmnCaseNo': '2018가단68430'},
 'assrs': {'acusrAssrs': ['피고는 원고들에게 000에 투자하면 한 달에 400만 원의 수익을 얻을 수 있고, 4개월이지나면 원금도 반환받을 수 있다고 설명하면서 원고들을 현혹하여 투자금을 유치하였다.',
   '000는 전형적인 금융피라미드 수법의사기업체로서 원고들이 피고의 설명과 같은 수익을 올리거나 투자금을 반환받는 것이 불가능하였다.',
   '피고는 이를 알면서도 위와 같이 원고들을 기망하여 000에 투자하도록 하였거나, 설령 그렇지 않더라도 원고들에게 000에 대한 투자의 위험성을 제대로 알리지 않은 과실이 있다. ',
   '따라서 피고는 고의 또는 과실에 의한 불법행위에 대한 손해배상으로 원고들이 입은 위 투자금 상당의 손해를 배상할 책임이 있다.'],
  'dedatAssrs': []},
 'facts': ['원고 A은 2017. 12. 31. 0000에 거주하는 친구 F을 만나러 갔다가 당시 F의 집에 있던 F의 동생인 피고를 만나 비트코인 투자업체인 000에 대한 설명을 듣고 투자를 결심하였다.',
  '나머지 원고들은 2018. 1. 6.경 0000소재 피고의 여동생이 운영하는 식당으로 피고를 만나러 가서 피고로부터 000에 대한 설명을 듣고 각 투자를 결심하였다.',
  "원고 A은 2017. 12. 31. 1

# 데이터 전처리

In [ ]:
from google.colab import drive
import json
# json 파일 경로
file_path_train = '/content/drive/MyDrive/Kaggle/6th-aikuthon/train.json'
file_path_test = '/content/drive/MyDrive/Kaggle/6th-aikuthon/test.json'

# json 파일 열기 및 파싱
trainset = pd.read_json(file_path_train)
testset = pd.read_json(file_path_test)

# 데이터 확인
print(trainset.head())

                                                info  \
0  {'caseField': '1', 'detailField': '1', 'trailF...   
1  {'caseField': '1', 'detailField': '1', 'trailF...   
2  {'caseField': '1', 'detailField': '1', 'trailF...   
3  {'caseField': '1', 'detailField': '1', 'trailF...   
4  {'caseField': '1', 'detailField': '1', 'trailF...   

                                       concerned  \
0                   {'acusr': '1', 'dedat': '1'}   
1  {'acusr': '1', 'dedat': '일본소속A~J,O,P주식회사, K'}   
2             {'acusr': '1', 'dedat': '국가, 자연인'}   
3                   {'acusr': '1', 'dedat': '2'}   
4                   {'acusr': '1', 'dedat': '2'}   

                                                 org  \
0  {'orgJdgmnCourtNm': '울산지방법원', 'orgJdgmnAdjuDe'...   
1  {'orgJdgmnCourtNm': '', 'orgJdgmnAdjuDe': '', ...   
2  {'orgJdgmnCourtNm': '', 'orgJdgmnAdjuDe': '', ...   
3  {'orgJdgmnCourtNm': '창원지방법원', 'orgJdgmnAdjuDe'...   
4  {'orgJdgmnCourtNm': '대구지방법원 서부지원', 'orgJdgmnAd...   

             

In [ ]:
def is_non_numeric(value):
    # 숫자 1, 2, 3, 4를 제외한 값이 있는지 확인
    return value not in ['1', '2', '3', '4', '5']
for concern in trainset['concerned']:
  if is_non_numeric(concern['dedat']):
    concern['dedat'] = 5
  if is_non_numeric(concern['acusr']):
    concern['acusr'] = 5


trainset['concerned'].head()

,concerned
0,"{'acusr': '1', 'dedat': '1'}"
1,"{'acusr': '1', 'dedat': 5}"
2,"{'acusr': '1', 'dedat': 5}"
3,"{'acusr': '1', 'dedat': '2'}"
4,"{'acusr': '1', 'dedat': '2'}"


In [ ]:
trainset['assrs'][0]
trainset['facts'][0]

trainset['label'][0]

1

In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel

class LegalPredictionModel(nn.Module):
    def __init__(self, bert_model_name='monologg/kobert', num_categories=10, hidden_size=64):
        super(LegalPredictionModel, self).__init__()

        # BERT 모델 로드
        self.bert = BertModel.from_pretrained(bert_model_name)

        # 범주형 데이터 처리용 MLP
        self.category_dense = nn.Linear(num_categories, hidden_size)
        self.relu = nn.ReLU()

        # 결합된 출력 처리용 레이어
        bert_hidden_size = self.bert.config.hidden_size
        self.classifier = nn.Linear(bert_hidden_size + hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask, category_input):
        # BERT 처리
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = bert_output.pooler_output

        # 범주형 데이터 처리
        category_output = self.relu(self.category_dense(category_input))

        # BERT 출력과 범주형 데이터 출력 결합
        combined_output = torch.cat((pooled_output, category_output), dim=1)

        # 최종 예측
        logits = self.classifier(combined_output)
        output = self.sigmoid(logits)

        return output

# 하이퍼파라미터 설정
max_len = 128
num_categories = 10  # 범주형 데이터의 차원

# 모델 초기화
model = LegalPredictionModel(num_categories=num_categories)

# 손실 함수와 옵티마이저 설정
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# 모델 요약
print(model)